# Standard Cuts Cross-section Analysis 




In [2]:
run ../prep_ana.py

Loading plotting specs...
8 or 13 TeV:
Loading 8TeV
unc_mc_process and scales as dictionaries
Load MC and Data?True
Correct MET?True
df = pd.concat([df_dy0, df_dy1, df_dy2, df_dy3, df_dy4, df_dy_m_10, df_ww, df_tt_l, df_tt_sl, df_zz_ln, df_wz_ln, df_wz_lq ])


In [3]:
%matplotlib inline

In [4]:
df_da_a = rp.read_root(data_path+"/muon_2012A_complete.root", columns=columns+["runNum", "eventNumb"])
df_da_b = rp.read_root(data_path+"/muon_2012B_complete.root", columns=columns+["runNum", "eventNumb"])
df_da_c = rp.read_root(data_path+"/muon_2012C_complete.root", columns=columns+["runNum", "eventNumb"])
df_da_d = rp.read_root(data_path+"/muon_2012D_complete.root", columns=columns+["runNum", "eventNumb"])

df_da = pd.concat([df_da_a, df_da_b, df_da_c, df_da_d]).reset_index()

In [5]:
df_da_a_el = rp.read_root(data_path+"/electron_2012A_complete.root", columns=columns+["runNum", "eventNumb"])
df_da_b_el = rp.read_root(data_path+"/electron_2012B_complete.root", columns=columns+["runNum", "eventNumb"])
df_da_c_el = rp.read_root(data_path+"/electron_2012C_complete.root", columns=columns+["runNum", "eventNumb"])
df_da_d_el = rp.read_root(data_path+"/electron_2012D_complete.root", columns=columns+["runNum", "eventNumb"])

df_da_el = pd.concat([df_da_a_el, df_da_b_el, df_da_c_el, df_da_d_el]).reset_index()
df_da = pd.concat( [df_da, df_da_el] ).reset_index()
df_da = df_da.drop(np.where(df_da[["runNum", "eventNumb"]].duplicated())[0])

In [ ]:
feature = "numb_jets"

mc = bin_df(cuts_ana(df), feature)
da = bin_df(cuts_ana(df_da), feature)

full_plot(mc, da, color="color_1")

# Plots

In [ ]:
flavor = 'same'
#flavor = 'diff'
create_kinematic_hist(cuts_ana(df, flavor=flavor), cuts_ana(df_da, flavor=flavor), prefix="8TeV/cuts_based/cuts_"+flavor)

# Yields

In [7]:
df_yields = process_yields(cuts_ana(df, ), cuts_ana(df_da) )
save_df_to_html( df_yields, "yields_may.html")

WW 1703.82176858 2339.64692465
DY 657.898419637 246.662307144
Top 1145.97243271 1511.74436154
WZ 85.4148814871 72.1567360398
ZZ 85.3103208289 3.4173310091
WG 224.258837126 63.0712423336
Higgs 42.9656817612 48.6474689591
{'Diff Flavor': [2217, 123, 2340, 247, 1512, 72, 3, 63, 49, 284, 4569, 5268], 'Same Flavor': [1602, 102, 1704, 658, 1146, 85, 85, 224, 43, 163, 4109, 4506], 'Process': ['WW', 'GluGluToWWTo4L', 'WW', 'DY', 'Top', 'WZ', 'ZZ', 'WG(*)', 'Higgs', 'WJ', 'Total', 'DATA']}


# Cross-sections

In [ ]:
def calc_cross_stuff(df_mc, df_data, flavor="both"):
    lumi = 19.7e3
    eff  = .1247 #this had to be changed previously
    Br   = (3*.108)**2. 
    
    df_mc_c = df_mc[df_mc.lep1_Charge != df_mc.lep2_Charge]
    df_data_c = df_data[df_data.lep1_Charge != df_data.lep2_Charge]
    
    df_mc_c_s = df_mc[df_mc.lep1_Charge == df_mc.lep2_Charge]
    df_data_c_s = df_data[df_data.lep1_Charge == df_data.lep2_Charge]
    
    N_mc = sum([ cuts_ana(df_mc_c[df_mc_c.process_decay == process], flavor=flavor).weight.values.sum()*scales[process] for process in scales.keys() if process not in ['WW', 'W1JetsToLNu','W2JetsToLNu','W3JetsToLNu','W3JetsToLNu'] ])
    N_Wjets = cuts_ana(df_data_c_s, flavor=flavor).shape[0] -  np.array([ cuts_ana(df_mc_c_s[(df_mc_c_s.process_decay == process) ], flavor=flavor).weight.values.sum()*scales[process] for process in scales.keys() if process not in [ 'W1JetsToLNu','W2JetsToLNu','W3JetsToLNu','W3JetsToLNu'] ]).sum()
    N_mc += N_Wjets
    
    N_data = cuts_ana(df_data_c, flavor=flavor).shape[0]
    
    N_ww_select = cuts_ana(df_mc_c[df_mc_c.process_decay == "WW"], flavor=flavor).weight.values.sum()*scales["WW"]
    N_ww_tot = df_mc[df_mc.process_decay == "WW"].weight.values.sum()*scales["WW"]

    ratio_s_t = N_ww_select / N_ww_tot
    #print "MC", N_mc
    #print "DATA", N_data
    #print N_ww_select / N_ww_tot
    #print "WJ", N_Wjets
    return {"lumi": lumi, "eff": eff, "Br": Br, "N_mc": N_mc, "N_data": N_data, "ratio_s_t": ratio_s_t, "N_ww_select":N_ww_select, "N_Wjets": N_Wjets}

In [ ]:
def stat_unc_calc(df_mc, df_data, flavor="both", fiducial=False, **kwargs):
    if kwargs:
        var = kwargs
    else:
        var = calc_cross_stuff(df_mc, df_data, flavor)
    lumi = var["lumi"]
    eff = var["eff"]
    Br = var["Br"]
    N_mc = var["N_mc"]
    N_data = var["N_data"]
    ratio_s_t = var["ratio_s_t"]
    N_ww_select = var["N_ww_select"]
    N_Wjets = var["N_Wjets"]
    
    cuts_mc = {process: cuts_ana(df_mc[(df_mc.process_decay == process) & (df_mc.lep1_Charge == df_mc.lep2_Charge)], flavor) for process in scales.keys()} 
    process_stat_unc = [ (cuts_mc[process].weight.values * cuts_mc[process].weight.values).sum()  * scales[process]**2 for process in cuts_mc.keys() if process not in ['WW', 'W1JetsToLNu','W2JetsToLNu','W3JetsToLNu','W3JetsToLNu']] 
    
    WW_stat_unc = (cuts_mc["WW"].weight.values * cuts_mc["WW"].weight.values).sum() * scales["WW"]**2. * ratio_s_t**2. / N_ww_select**2.
    
    #print "Process stat", process_stat_unc
    #print "WW stat",WW_stat_unc
    if fiducial == False:
        return 1. / (lumi * eff * Br * ratio_s_t) * ( N_data + sum(process_stat_unc) + WW_stat_unc + N_Wjets)**.5
    else:
        return 1. / (lumi * eff ) * ( N_data + sum(process_stat_unc) + N_Wjets)**.5

In [ ]:
def sys_unc_calc(df_mc, df_data, flavor="both", fiducial=False, **kwargs):
    if kwargs:
        var = kwargs
    else:
        var = calc_cross_stuff(df_mc, df_data, flavor)
    lumi = var["lumi"]
    eff = var["eff"]
    Br = var["Br"]
    N_mc = var["N_mc"]
    N_data = var["N_data"]
    ratio_s_t = var["ratio_s_t"]
    N_ww_select = var["N_ww_select"]
    N_Wjets = var["N_Wjets"]
    
    cuts_mc = {process: cuts_ana(df_mc[(df_mc.process_decay == process) & (df_mc.lep1_Charge == df_mc.lep2_Charge)], flavor) for process in scales.keys()} 
    process_sys_unc = [ scales[process]**2. * unc_mc_process[process]**2 * cuts_mc[process].weight.sum()**2. for process in cuts_mc.keys() if process not in ['WW', 'W1JetsToLNu','W2JetsToLNu','W3JetsToLNu','W3JetsToLNu']] 

    WW_sys_unc = unc_mc_process["WW"]**2 * ratio_s_t**2. / N_ww_select**2.
    Wjets_sys_unc = N_Wjets**2. 
    
    if fiducial == False:
        return 1. / (lumi * eff * Br * ratio_s_t)  * ( sum(process_sys_unc) +  WW_sys_unc + Wjets_sys_unc)**.5
    else:
        return 1. / (lumi * eff )  * ( sum(process_sys_unc) + Wjets_sys_unc)**.5

In [ ]:
def cross_calc(df_mc, df_data, flavor="both", fiducial=False, **kwargs):
    if kwargs:
        var = kwargs
        print "Got them words"
    else:
        var = calc_cross_stuff(df_mc, df_data, flavor)
    lumi = var["lumi"]
    eff = var["eff"]
    Br = var["Br"]
    N_mc = var["N_mc"]
    N_data = var["N_data"]
    ratio_s_t = var["ratio_s_t"]
    N_ww_select = var["N_ww_select"]
    N_Wjets = var["N_Wjets"]
    
    if fiducial == False:
        return (N_data - N_mc) / (lumi * eff * Br *ratio_s_t)
    else:
        return (N_data - N_mc) / (lumi * eff )
    #return N_ww_select / (lumi * eff * Br *ratio_s_t)

In [ ]:
full = {}

flavor = 'both'
kwds = calc_cross_stuff(df, df_da, flavor)
full['both'] = [ cross_calc(df, df_da, flavor=flavor, **kwds), stat_unc_calc(df, df_da, flavor, **kwds), sys_unc_calc(df, df_da, flavor, **kwds)]
print full['both']

flavor = 'same'
kwds = calc_cross_stuff(df, df_da, flavor)
full['same'] = [cross_calc(df, df_da, flavor=flavor, **kwds), stat_unc_calc(df, df_da, flavor, **kwds), sys_unc_calc(df, df_da, flavor, **kwds)]
print full['same']

flavor = 'diff'
kwds = calc_cross_stuff(df, df_da, flavor)
full['diff'] = [ cross_calc(df, df_da, flavor=flavor, **kwds), stat_unc_calc(df, df_da, flavor, **kwds), sys_unc_calc(df, df_da, flavor, **kwds)]
print full['diff']

In [ ]:
fiducial = {}

flavor = 'both'
kwds = calc_cross_stuff(df, df_da, flavor)
fiducial['both'] = [ cross_calc(df, df_da, flavor=flavor, fiducial=True, **kwds), stat_unc_calc(df, df_da, flavor, fiducial=True, **kwds), sys_unc_calc(df, df_da, flavor, fiducial=True, **kwds)]
print flavor, fiducial['both']

flavor = 'same'
kwds = calc_cross_stuff(df, df_da, flavor)
fiducial['same'] = [cross_calc(df, df_da, flavor=flavor, fiducial=True, **kwds), stat_unc_calc(df, df_da, flavor, fiducial=True, **kwds), sys_unc_calc(df, df_da, flavor, fiducial=True, **kwds)]
print flavor, fiducial['same']

flavor = 'diff'
kwds = calc_cross_stuff(df, df_da, flavor)
fiducial['diff'] = [ cross_calc(df, df_da, flavor=flavor, fiducial=True, **kwds), stat_unc_calc(df, df_da, flavor, fiducial=True, **kwds), sys_unc_calc(df, df_da, flavor, fiducial=True, **kwds)]
print flavor, fiducial['diff']


In [ ]:
pd.DataFrame(fiducial)

# Make cross-section table

In [ ]:
from html import HTML

In [ ]:
html_bot = HTML('html', 'text')

In [ ]:
def make_cross_table(cross_dic):
    html_bot = HTML('html', 'text')
    t = html_bot.table( border="0", border_="collapse")
    key_syn = {"both": "Together", "same": "Same", "diff": "Different"}
    for key in cross_dic:
        r = t.tr
        for i, ele in enumerate(cross_dic[key]):
            prefix = ""
            postfix = ""
            if i == 0: r.td.b(key_syn[key]+":" , border="none")
            else: 
                prefix = "&plusmn;"
                if i == 1: postfix = "(stat)"
                if i == 2: postfix = "(sys)"
            r.td(prefix + str(round(ele, 2)) + postfix, border="none", escape=False)
    print t


In [ ]:
make_cross_table(fiducial)

In [ ]:
print t

<table border="0" border_="collapse">
<tr><td><b border="none">both:</b></td><td border="none">1.36</td><td border="none">&plusmn;0.03</td><td border="none">&plusmn;0.05</td></tr>
<tr><td><b border="none">same:</b></td><td border="none">0.54</td><td border="none">&plusmn;0.02</td><td border="none">&plusmn;0.01</td></tr>
<tr><td><b border="none">diff:</b></td><td border="none">0.82</td><td border="none">&plusmn;0.03</td><td border="none">&plusmn;0.04</td></tr>
</table>




In [ ]:
html_bot.table()

In [ ]:
round( ele, 2)

# lep, met, jet shifts

In [ ]:
def scale_shift(data, lep_pt=1.01, met=1.02, jet=1.02 ):
    data.lep1_pt = data.lep1_pt * lep_pt
    data.lep2_pt = data.lep2_pt * lep_pt
    data.mll = data.mll * lep_pt**.5
    
    
    data.metMod = data.metMod * met
    data.metProj = data.METProj * met
    
    data.mllMET = data.mllMET * met * lep_pt**.5
    
    
    data.HT = data.HT * jet

In [ ]:
scale_shift(df_da)

In [ ]:
bin_df?

In [ ]:
df[df.lep1_pt > 200].lep_Type.unique()

In [ ]:
mc = bin_df(df[(df.lep1_pt > 200)], "weight", range=(-.25,2), bins=100)
data = bin_df(df_da[(df_da.lep1_pt > 200) ], "weight", range=(-.25,2), bins=100)

full_plot(mc, data, )

In [ ]:
feature = "numb_jets"

bins_mc = bin_df( cuts_ana(df[(df.lep1_Charge != df.lep2_Charge) ]), feature)
bins_data = bin_df( cuts_ana(df_da[(df_da.lep1_Charge != df_da.lep2_Charge) ]), feature)

fig, ax = full_plot(bins_mc, bins_data, color="color_1", )#logy=False, y_range=(0,12500) )
pre_df = pre_df = pre_cuts(df, diff_charge=False)
ax[0].hist(pre_df[df.process == "WW"].numb_jets.values, weights=pre_df[pre_df.process == "WW"].weight.values * scales["WW"], range=(-.5, 9.5), histtype=u'step', color="black", label="MC WW")
